In [1]:
import numpy as np
from lxml import objectify

import mbuild as mb
import foyer
import ele

from foyer_xml_writer import parmed_to_foyer_xml

In [3]:
methane = mb.load("CCCC", smiles=True)
ff = foyer.forcefields.get_forcefield(name="opls")
methane_pmd = ff.apply(methane, assert_dihedral_params=True)

/home/chris/miniconda3/envs/gmso-dev/lib/python3.10/site-packages/foyer/validator.py:165: ValidationWarning: You have empty smart definition(s)
  warn("You have empty smart definition(s)", ValidationWarning)
/home/chris/cme/forks/mbuild/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([0.1936 0.0065 0.0247]), 4 bonds, id: 139686099938096> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/chris/cme/forks/mbuild/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([ 0.0537  0.0185 -0.0351]), 4 bonds, id: 139689118381440> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/chris/cme/forks/mbuild/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.0537 -0.0185  0.0677]), 4 bonds, id: 139686099933920> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/chris/cme/forks/mbuild/mbuild/compound.py:400: UserWarning: Some particle of <C p

In [4]:
parmed_to_foyer_xml(structure=methane_pmd, ff=ff, file_name="test.xml")

In [ ]:
def write_atom_type(name, atom_type, element, mass, _def=None, desc=None):
    line = f'\t\t<Type name="{name}" class="{atom_type}" element="{element}" mass="{mass}" def="{_def}" desc="{desc}"/>\n'
    return line


def write_harmonic_bond(class1, class2, l0, k):
    line = f'\t\t<Bond class1="{class1}" class2="{class2}" length="{l0}" k="{k}"/>\n'
    return line


def write_harmonic_angle(class1, class2, class3, t0, k):
    line = f'\t\t<Angle class1="{class1}" class2="{class2}" class3="{class3}" angle="{t0}" k="{k}"/>\n'
    return line


def write_non_bonded(name, charge, sigma, epsilon):
    line = f'\t\t<Atom type="{name}" charge="{charge}" sigma="{sigma}" epsilon="{epsilon}"/>\n'
    return line


def write_rb_torsion(class1, class2, class3, class4, c0, c1, c2, c3, c4, c5):
    line = f'\t\t<Proper class1="{class1}" class2="{class2}" class3="{class3}" class4="{class4}" c0="{c0}" c1="{c1}" c2="{c2}" c3="{c3}" c4="{c4}" c5="{c5}"/>\n'
    return line


def write_dihedral():
    pass


def get_atom_types(structure):
    return tuple(set(a.type for a in structure.atoms))


def get_bond_types(structure):
    bond_types = set()
    for i in structure.bonds:
        bond_types.add((i.atom1.type, i.atom2.type))
    return tuple(bond_types)


def get_angle_types(structure):
    angle_types = set()
    for i in structure.angles:
        angle_types.add((i.atom1.type, i.atom2.type, i.atom3.type))
    return tuple(angle_types)


def get_dihedral_types(structure):
    """Use this if the parmed structure was created using the GAFF forcefield"""
    dihedral_types = set()
    for i in structure.dihedrals:
        dihedral_types.add(
            (i.atom1.type, i.atom2.type, i.atom3.type, i.atom4.type)
        )
    return tuple(dihedral_types)


def get_rb_tortions(structure):
    """Use this if the parmed structure was created using the OPLS forcefield"""
    dihedral_types = set()
    for i in structure.rb_torsions:
        dihedral_types.add(
            (i.atom1.type, i.atom2.type, i.atom3.type, i.atom4.type)
        )
    return tuple(dihedral_types)

In [ ]:
def parmed_to_foyer_xml(structure, ff, file_name):
    with open(file_name, "w") as f:
        f.write(f'<ForceField name="{ff.name}" version="{ff.version}" combining_rule="{ff.combining_rule}">\n')
        f.write("\t<AtomTypes>\n")
        for atom in get_atom_types(structure):
            atom_type = ff.atomTypeClasses[atom]
            element=ff.atomTypeElements[atom]
            mass = ele.element_from_symbol(element).mass
            _def = ff.atomTypeDefinitions[atom]
            desc = ff.atomTypeDesc[atom]
            line = write_atom_type(
                name=atom,
                atom_type=ff.atomTypeClasses[atom],
                element=ff.atomTypeElements[atom],
                mass=mass,
                _def=_def,
                desc=desc
            )
            f.write(line)
        f.write("\t</AtomTypes>\n")
        
        f.write("\t<HarmonicBondForce>\n")
        for bond in get_bond_types(structure):
            params = ff.get_parameters("harmonic_bonds", bond)
            class1 = ff.atomTypeClasses[bond[0]]
            class2 = ff.atomTypeClasses[bond[1]]
            line = write_harmonic_bond(class1=class1, class2=class2, l0=params["length"], k=params["k"])
            f.write(line)
        f.write("\t</HarmonicBondForce>\n")
        
        f.write("\t<HarmonicAngleForce>\n")
        for angle in get_angle_types(structure):
            params=ff.get_parameters("harmonic_angles", angle)
            class1 = ff.atomTypeClasses[angle[0]]
            class2 = ff.atomTypeClasses[angle[1]]
            class3 = ff.atomTypeClasses[angle[2]]
            line = write_harmonic_angle(
                class1=class1, class2=class2, class3=class3, t0=params["theta"], k=params["k"]
            )
            f.write(line)
        f.write("\t</HarmonicAngleForce>\n")
        
        if "OPLS" in ff.name:
            f.write("\t<RBTorsionForce>\n")
            for dihedral in get_rb_tortions(structure):
                params = ff.get_parameters("rb_propers", dihedral)
                class1 = ff.atomTypeClasses[dihedral[0]]
                class2 = ff.atomTypeClasses[dihedral[1]]
                class3 = ff.atomTypeClasses[dihedral[2]]
                class4 = ff.atomTypeClasses[dihedral[3]]
                line = write_rb_torsion(
                    class1=class1,
                    class2=class2,
                    class3=class3,
                    class4=class4,
                    c0=params["c0"],
                    c1=params["c1"],
                    c2=params["c2"],
                    c3=params["c3"],
                    c4=params["c4"],
                    c5=params["c5"],
                )
                f.write(line)
            f.write("\t</RBTorsionForce>\n")
        
        f.write(f'\t<NonbondedForce coulomb14scale="{ff.coulomb14scale}" lj14scale="{ff.lj14scale}">\n')
        for atom in get_atom_types(structure):
            params = ff.get_parameters("atoms", atom)
            line = write_non_bonded(
                name=atom, charge=params["charge"], sigma=params["sigma"], epsilon=params["epsilon"]
            )
            f.write(line)
        f.write('\t</NonbondedForce>\n')
        f.write('</ForceField>')

In [ ]:
parmed_to_foyer_xml(methane_pmd, ff=ff, file_name="testing.xml")

In [ ]:
ff_test = foyer.Forcefield(forcefield_files="test.xml")

In [ ]:
methane_pmd2 = ff_test.apply(methane)